In [1]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from requests_html import HTML

In [2]:
def filter_by_response(response) -> bool:
    return ('\"incomplete_results\":true' not in response.text)
requests_cache.install_cache(expire_after=None, filter_fn=filter_by_response)

________________________________________

**Lấy tất cả tracks của một user**

Khởi tạo các list để lưu trữ các user, track, playlist.

In [3]:
users = []
tracks = []
playlists = []

Bắt đầu parse SoundCloud bằng trang Discover của SoundCloud vì trang này có nhiều playlists và trong đó có rất nhiều users và tracks thuận tiện cho việc parsing.

In [4]:
url = "https://soundcloud.com/discover"

Mở Chrome bằng webdriver và vào trang Discover của SoundCloud

In [5]:
browser = webdriver.Chrome()
browser.get(url)

In [6]:
soup = BeautifulSoup(browser.page_source)

Đồng ý Cookies

In [7]:
cookies = browser.find_element(By.XPATH,"//button[@id='onetrust-accept-btn-handler']")
cookies.click()

In [8]:
# Chọn một tab genre (demo)
time.sleep(2)
genre = browser.find_element(By.XPATH,"//a[@class='playableTile__heading playableTile__mainHeading sc-truncate sc-type-light sc-text-secondary sc-font-light sc-link-dark sc-link-primary sc-text-h4']")
genre.click()

Hàm click_till_end sử dụng để click vào button đến khi không còn để click.
Bởi vì trong trang Discover, sẽ bao gồm rất nhiều nhóm playlist có cùng thể loại (vd: Top 50, New & Hot, Study, v.v).
Mỗi nhóm có nhiều playlist khác nhau nên sẽ có 1 button (có hình mũi tên) khi nhấn vào sẽ chuyển qua các playlist khác.

In [ ]:
# test not done
def click_till_end():
    while True:
        elm = browser.find_element_by_tag_name('button')
        if 'inactive' in elm.get_attribute('class'):
            break;
        elm.click()

Hàm scroll_down_till_end sử dụng để scroll trang xuống hết mức có thể.
Khi vào 1 trang (vd: trang Profile của 1 user), ta chỉ xem được 1 phần đầu của trang đó.
Khi scroll xuống sẽ xuất hiện thêm thông tin để ta parse.

In [9]:
# nên check lại bởi vì vài lần chưa scroll hết :<
def scroll_down_till_end():
    # Loop 2 lần cho chắc ăn (có thể cho 3 lần)
    for i in range(2):
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        lastCount = 0
        match = False
        
        while(match == False):
            time.sleep(3)
            lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount == lenOfPage: 
                match = True
            else: lastCount = lenOfPage

Hàm collect_tracks_playlist sử dụng để lấy hết tất cả track và user có trong 1 playlist.

In [10]:
def collect_tracks_playlist():
    scroll_down_till_end()
    
    time.sleep(2)
    soup = BeautifulSoup(browser.page_source)
    
    posts = soup.find_all("div", "trackItem__content")

    for post in posts:
        user = post.find("a", "trackItem__username").get_text()
        # kiểm tra có bị trùng
        if user not in users:
            users.append(user)

        track = post.find("a", "trackItem__trackTitle").get_text()
        # kiểm tra có bị trùng
        if track not in tracks:
            tracks.append(track)

Hàm collect_tracks_user_profile sử dụng để lấy tất cả track của user đó.

In [11]:
def collect_tracks_user_profile():
    scroll_down_till_end()
            
    time.sleep(3)
    soup = BeautifulSoup(browser.page_source)
    
    songs = soup.find_all("li", "soundList__item")
    
    for track in songs:
        title = track.find("a", "soundTitle__title").get_text().strip()
        # kiểm tra có bị trùng
        if title not in tracks:
            tracks.append(title)

In [12]:
collect_tracks_playlist()

In [13]:
time.sleep(2)
profile = browser.find_element(By.XPATH,"//a[@class='trackItem__username sc-link-light sc-link-secondary']")
profile.click()

In [20]:
collect_tracks_user_profile()

In [23]:
print(tracks)
print(len(tracks))

['Tell Ur Mom II - Winno (ft Heily)', 'PHI HÀNH GIA - RENJA x SLOW T x LIL WUYN x KAIN x SUGAR CANE', 'Lạc - Rhymastic', 'Nếu mai chia tay ( Hieu Tran Chill RnB Version) - Monstar Ft. Amee', 'LeeHi (이하이) - ONLY', 'GPS - NIDEBIG ft BINQANG , PURR ( PROD : HS YAFU )', 'không phải tại nó // lạc - mai ngô ft rhymastic', '[DCOD] NGTANOISE (Người ta nói - remake) | VSOUL x MFREE x TUYEN VO|', 'Sài Gòn Đâu Có Lạnh - Changg x Lewiuy', 'HIEUTHUHAI - NGHE NHƯ TÌNH YÊU (FELIKS ALVIN REMIX) | HAWYS EDIT', 'Biết em - Lynk Nguyễn', 'Cưa Là Đổ - Phát Hồ X2X (Đại Mèo Remix)', '3107-3 (lofi ver. by VUX) - W/n x Nâu x Duongg x Titie', 'Việt - Hơn Cả Mây Trời (lofi ver by hawys)', 'Orange x Hoàng Dũng - Khi Em Lớn (Lo-fi ver by Hawys)', 'Obito Ft. Hnhngan Live YOUR SMILE - Emma X Seachains X Obito', 'Tell Ur Mom II (Remix)Winno ft Heily', 'Ở NHÀ CHÁN QUÓ (Prod. KanaBeats)', 'Tlinh - Thích Quá Rùi Nà ft. Trung Trần (prod by @rpt.phongkhin )', 'Khi co don em nho ai', 'THANH XUÂN LÀ ANH - N.Dien X 2T f.t 

In [19]:
# lùi 1 trang
browser.execute_script("window.history.go(-1)")